<a href="https://colab.research.google.com/github/Huni1i1/Running-Posture-Correction-AI/blob/main/RPC_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# Check the gpu
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Install Detectron2
!pip3 install -q torch
!pip3 install -q torchvision
!pip3 install pyyaml
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-50yddvqg
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-50yddvqg
  Resolved https://github.com/facebookresearch/detectron2.git to commit 754469e176b224d17460612bdaa2cb8112b04cd9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.7 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [2]:
# Check the version
import torch, detectron2, os
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

ModuleNotFoundError: No module named 'detectron2'

In [2]:
# Clone VideoPose3D
import os

os.chdir('/content')
!git clone https://github.com/facebookresearch/VideoPose3D

Cloning into 'VideoPose3D'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121 (from 1)
Receiving objects: 100% (121/121), 9.53 MiB | 28.78 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
# Change infer_video_d2.py and run.py
import shutil

os.chdir('/content')
!git clone https://github.com/Huni1i1/RPC_requirements
shutil.move('/content/RPC_requirements/infer_video_d2.py', '/content/VideoPose3D/inference/infer_video_d2.py')
shutil.move('/content/RPC_requirements/run.py', '/content/VideoPose3D/run.py')

Cloning into 'RPC_requirements'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 90 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 804.53 KiB | 3.89 MiB/s, done.
Resolving deltas: 100% (13/13), done.


'/content/VideoPose3D/run.py'

In [4]:
# Download pre-trained model
os.makedirs('/content/VideoPose3D/checkpoint', exist_ok = True)
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin -P /content/VideoPose3D/checkpoint

--2024-11-20 10:49:19--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.165.160.106, 3.165.160.70, 3.165.160.120, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.165.160.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘/content/VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M   143MB/s    in 0.5s    

2024-11-20 10:49:19 (143 MB/s) - ‘/content/VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



# Dataset

In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/videos', exist_ok = True)
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/Det2', exist_ok = True)
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/VP3D_joint', exist_ok = True)
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/VP3D_render', exist_ok = True)
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/angles', exist_ok = True)
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/labels', exist_ok = True)
os.makedirs('/content/drive/MyDrive/CapstoneDesign/train/checkpoint', exist_ok = True)

In [7]:
ORIGINAL_VIDEOS = '/content/drive/MyDrive/CapstoneDesign/train/original_videos'

VIDEOS = '/content/drive/MyDrive/CapstoneDesign/train/videos'
DET2 = '/content/drive/MyDrive/CapstoneDesign/train/Det2'
VP3D_JOINT = '/content/drive/MyDrive/CapstoneDesign/train/VP3D_joint'
VP3D_RENDER = '/content/drive/MyDrive/CapstoneDesign/train/VP3D_render'
ANGLES = '/content/drive/MyDrive/CapstoneDesign/train/angles'
LABELS = '/content/drive/MyDrive/CapstoneDesign/train/labels'
CHECKPOINT = '/content/drive/MyDrive/CapstoneDesign/train/checkpoint'

In [10]:
# # Crop videos to square
# import os
# import glob
# import subprocess

# def crop_video_to_square(input_folder, output_folder):
#     video_files = sorted(glob.glob(os.path.join(input_folder, '*.mp4')))

#     for input_video_path in video_files:
#         output_video_path = os.path.join(output_folder, os.path.basename(input_video_path))

#         print(f"Cropping video: {input_video_path} -> {output_video_path}")

#         command = [
#         'ffmpeg', '-i', input_video_path,
#         '-vf', 'crop=min(in_w\,in_h):min(in_w\,in_h)',
#         '-c:a', 'copy',
#         output_video_path
#         ]

#         subprocess.run(command)

#     print(f"All videos have been cropped and saved in {output_folder}.")

# crop_video_to_square(ORIGINAL_VIDEOS, VIDEOS)

Cropping video: /content/drive/MyDrive/dataset/original_videos/1_0.mp4 -> /content/data/videos/1_0.mp4
Cropping video: /content/drive/MyDrive/dataset/original_videos/1_1.mp4 -> /content/data/videos/1_1.mp4
Cropping video: /content/drive/MyDrive/dataset/original_videos/2_0.mp4 -> /content/data/videos/2_0.mp4
Cropping video: /content/drive/MyDrive/dataset/original_videos/2_1.mp4 -> /content/data/videos/2_1.mp4
All videos have been cropped and saved in 'VIDEOS'.


In [10]:
# Resample videos to 50fps
import os
import glob
import subprocess

def resample_videos_to_50fps(input_folder, output_folder):
    video_files = sorted(glob.glob(os.path.join(input_folder, '*.mp4')))

    for input_video_path in video_files:
        output_video_path = os.path.join(output_folder, os.path.basename(input_video_path))

        print(f"Resample video to 50fps: {input_video_path} -> {output_video_path}")

        command = [
            "ffmpeg",
            "-i", input_video_path,
            "-filter:v", "fps=fps=50",
            "-c:a", "copy",
            output_video_path
        ]

        subprocess.run(command, check=True)

    print(f"All videos have been resampled and saved in {output_folder}.")

resample_videos_to_50fps(ORIGINAL_VIDEOS, VIDEOS)

Resample video to 50fps: /content/drive/MyDrive/CapstoneDesign/train/original_videos/0_00.mp4 -> /content/drive/MyDrive/CapstoneDesign/train/videos/0_00.mp4
Resample video to 50fps: /content/drive/MyDrive/CapstoneDesign/train/original_videos/0_01.mp4 -> /content/drive/MyDrive/CapstoneDesign/train/videos/0_01.mp4
Resample video to 50fps: /content/drive/MyDrive/CapstoneDesign/train/original_videos/0_02.mp4 -> /content/drive/MyDrive/CapstoneDesign/train/videos/0_02.mp4
Resample video to 50fps: /content/drive/MyDrive/CapstoneDesign/train/original_videos/0_03.mp4 -> /content/drive/MyDrive/CapstoneDesign/train/videos/0_03.mp4
Resample video to 50fps: /content/drive/MyDrive/CapstoneDesign/train/original_videos/0_04.mp4 -> /content/drive/MyDrive/CapstoneDesign/train/videos/0_04.mp4
Resample video to 50fps: /content/drive/MyDrive/CapstoneDesign/train/original_videos/0_05.mp4 -> /content/drive/MyDrive/CapstoneDesign/train/videos/0_05.mp4
Resample video to 50fps: /content/drive/MyDrive/CapstoneDe

# Infer 2D Keypoints

In [11]:
# Infer 2D keypoints with Detectron2 and prepare custom dataset
def infer_Det2_and_prepare_dataset(video_folder, output_folder):
    mp4_files = sorted(glob.glob(os.path.join(video_folder, '*.mp4')))

    for mp4_file in mp4_files:
        print(f"Inferring 2D keypoints of {mp4_file}")

        os.chdir('/content/VideoPose3D/inference')
        subprocess.run(['python', 'infer_video_d2.py',
                        '--cfg', 'COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml',
                        '--output-dir', output_folder,
                        '--image-ext', 'mp4',
                        mp4_file])

        os.chdir('/content/VideoPose3D/data')
        subprocess.run(['python', 'prepare_data_2d_custom.py',
                        '-i', output_folder,
                        '-o', os.path.splitext(os.path.basename(mp4_file))[0]])

        for npz_file in glob.glob(os.path.join(output_folder, '*.npz')):
          os.remove(npz_file)

infer_Det2_and_prepare_dataset(VIDEOS, DET2)

Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_00.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_01.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_02.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_03.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_04.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_05.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_06.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_07.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_08.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_09.mp4
Inferring 2D keypoints of /content/drive/MyDrive/CapstoneDesign/train/videos/0_10.mp4
Inferring 2D keypoints of /content/drive/MyDrive/Capst

# Infer 3D Keypoints

In [8]:
shutil.rmtree('/content/VideoPose3D/data')
shutil.copytree('/content/drive/MyDrive/CapstoneDesign/train/data', '/content/VideoPose3D/data')

'/content/VideoPose3D/data'

In [12]:
# Infer 3D keypoints with VideoPose3D
def infer_VP3D(npz_folder, joint_folder):
    npz_files = sorted(glob.glob(os.path.join(npz_folder, '*.npz')))

    for npz_file in npz_files:
        video_name = os.path.splitext(os.path.basename(npz_file))[0]
        video_name = video_name[15:]

        print(f"Inferring 3D keypoints of {npz_file}")

        os.chdir('/content/VideoPose3D')
        subprocess.run(['python', 'run.py',
            '-d', 'custom',
            '-k', f'{video_name}',
            '-arc', '3,3,3,3,3',
            '-c', 'checkpoint',
            '--evaluate', 'pretrained_h36m_detectron_coco.bin',
            '--render',
            '--viz-subject', f'{video_name}.mp4',
            '--viz-action', 'custom',
            '--viz-camera', '0',
            '--viz-size', '10',
            '--viz-export', f'{joint_folder}/{video_name}_joint'
            ])

npz_folder = '/content/VideoPose3D/data'

infer_VP3D(npz_folder, VP3D_JOINT)

Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_00.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_01.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_02.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_03.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_04.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_05.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_06.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_07.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_08.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_09.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_10.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_11.npz
Inferring 3D keypoints of /content/VideoPose3D/data/data_2d_custom_0_12.npz
Inferring 3D

In [ ]:
# Render 3D keypoints with VideoPose3D
def render_VP3D(npz_folder, videos_folder, render_folder):
    npz_files = sorted(glob.glob(os.path.join(npz_folder, '*.npz')))

    for npz_file in npz_files:
        video_name = os.path.splitext(os.path.basename(npz_file))[0]
        video_name = video_name.replace('data_2d_custom_', '')

        print(f"Rendering {npz_file} with video name '{video_name}'...")

        os.chdir('/content/VideoPose3D')

        render_command = [
            'python', 'run.py',
            '-d', 'custom',
            '-k', f'{video_name}',
            '-arc', '3,3,3,3,3',
            '-c', 'checkpoint',
            '--evaluate', 'pretrained_h36m_detectron_coco.bin',
            '--render',
            '--viz-subject', f'{video_name}.mp4',
            '--viz-action', 'custom',
            '--viz-camera', '0',
            '--viz-size', '10',
            '--viz-video', f'{videos_folder}/{video_name}.mp4',
            '--viz-output', f'{render_folder}/{video_name}_render.mp4'
        ]

        subprocess.run(render_command, check=True)

        os.remove('/content/VideoPose3D/data/data_2d_custom_'+video_name+'.npz')


npz_folder = '/content/VideoPose3D/data'

render_VP3D(npz_folder, VIDEOS, VP3D_RENDER)

Rendering /content/VideoPose3D/data/data_2d_custom_3_02.npz with video name '3_02'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_03.npz with video name '3_03'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_04.npz with video name '3_04'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_05.npz with video name '3_05'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_06.npz with video name '3_06'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_07.npz with video name '3_07'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_08.npz with video name '3_08'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_09.npz with video name '3_09'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_10.npz with video name '3_10'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_11.npz with video name '3_11'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_12.npz with video name '3_12'...
Rendering /content/VideoPose3D/data/data_2d_custom_3_1

# Prepare Dataset and Labeling

In [ ]:
os.chdir('/content/RPC_requirements')
import angle, dl

In [ ]:
# Get angle dataset
def process_files(joints_folder, angles_folder):
    npy_files = sorted(glob.glob(os.path.join(joints_folder, '*.npy')))

    for file_path in npy_files:
        keypoints_data = np.load(file_path)
        print(f"Processing {file_path}...")

        processor = AngleProcessor(keypoints_data)
        angles = processor.getAngle()

        output_file_name = os.path.basename(file_path).replace('.npy', '_angles.npy')
        output_path = os.path.join(angles_folder, output_file_name)
        np.save(output_path, angles)

        print(f"Saved angles to {output_path}")


process_files(VP3D_JOINT, ANGLES)

In [ ]:
# Labeling
import numpy as np

label_mapping = {
    0: "Correct Posture",
    1: "Leaning Forward",
    2: "Leaning Backward",
    3: "Excessive Movement"
}

def generate_labels(angles_folder, labels_folder):
    angles_files = sorted(glob.glob(os.path.join(angles_folder, '*.npy')))

    for angles_file in angles_files:
        base_name = os.path.basename(angles_file)
        label = int(base_name.split('_')[0])

        angles_data = np.load(angles_file)
        num_frames = angles_data.shape[0]

        labels = np.full((num_frames,), label, dtype=np.int32)

        label_file_name = base_name.replace('.npy', '_labels.npy')
        label_file_path = os.path.join(labels_folder, label_file_name)

        np.save(label_file_path, labels)

        print(f"Label file saved: {label_file_path}")

generate_labels(ANGLES, LABELS)

Labeling /content/data/VP3D/0_0_joint.npy
라벨링할 숫자를 입력하세요: 0
Labeling /content/data/VP3D/0_1_joint.npy
라벨링할 숫자를 입력하세요: 0
Labeling /content/data/VP3D/1_0_joint.npy
라벨링할 숫자를 입력하세요: 1
Labeling /content/data/VP3D/1_1_joint.npy
라벨링할 숫자를 입력하세요: 1
Labeling /content/data/VP3D/2_0_joint.npy
라벨링할 숫자를 입력하세요: 2
Labeling /content/data/VP3D/2_1_joint.npy
라벨링할 숫자를 입력하세요: 2
Labeling /content/data/VP3D/3_0_joint.npy
라벨링할 숫자를 입력하세요: 3
Labeling /content/data/VP3D/3_1_joint.npy
라벨링할 숫자를 입력하세요: 3
Labeling /content/data/VP3D/4_0_joint.npy
라벨링할 숫자를 입력하세요: 4
Labeling /content/data/VP3D/4_1_joint.npy
라벨링할 숫자를 입력하세요: 4


# Train


In [ ]:
# Train the model
angles_files = sorted(glob.glob(f'{ANGLES}/*.npy'))
labels_files = sorted(glob.glob(f'{LABELS}/*.npy'))

input_size = 15
batch_size = 4
dataset = PostureDataset(angles_files, labels_files)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: (
    torch.cat([item[0] for item in x], dim=0),
    torch.cat([item[1] for item in x], dim=0)
))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PostureClassifierFCNN(input_size=input_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, dataloader, criterion, optimizer, device, num_epochs=20)

model_save_path = CHECKPOINT + '/model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model training completed and saved at {model_save_path}.")